<a href="https://colab.research.google.com/github/adimnicky/Capstone-Project/blob/main/Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_community
!pip install replicate
!pip install pandas

In [ ]:
from langchain_community.llms import Replicate
import os
from google.colab import userdata
import pandas as pd

# Set the API token
api_token = userdata.get('api_token')
os.environ["REPLICATE_API_TOKEN"] = api_token
# Model setup
model = "ibm-granite/granite-3.3-8b-instruct"
output = Replicate(
model=model,
replicate_api_token=api_token,
)

# Baca file CSV
df = pd.read_csv("Tweets.csv")

# Tampilkan nama kolom untuk memastikan kolom teks
print(df.columns)


In [ ]:
# Fungsi untuk memecah list menjadi beberapa batch
def chunk_reviews(reviews, batch_size):
    for i in range(0, len(reviews), batch_size):
        yield reviews[i:i + batch_size]

# Batasi 10 review per batch
batch_size = 10
batches = list(chunk_reviews(customer_reviews, batch_size))

# Refine the prompt to include reviews
for idx, batch in enumerate(batches):
    reviews_text = "\n".join([f"Review {i+1}: {review}" for i, review in enumerate(batch)])
    prompt = f"""
Classify these reviews as Positive, Negative, or Mixed:
{reviews_text}
"""
# Invoke the model with the example prompt
response = output.invoke(prompt)

# Print the response
print(f"\n--- Granite Model Response: Batch {idx+1} ---\n")
print(response)

In [ ]:
# Define refined prompt
refined_prompt = f"""
Classify these reviews as positive, negative, or mixed, and tag
relevant categories (late flight, or rude service):
{reviews_text}
"""
# Invoke the model with the example prompt
response = output.invoke(refined_prompt)
# Print the response
print("Granite Model Refined Response:\n")
print(response)

In [ ]:
# Define the prompt to complete the task in 2 steps
multitask_prompt = f"""
Complete the task in 2 steps.
Step 1: Classify these reviews as positive, negative, or mixed.
Step 2: For each review, identify relevant categories: battery
life, screen quality, or performance.
{reviews_text}
"""
response = output.invoke(multitask_prompt)
print("Granite Model Response:\n")
print(response)

In [ ]:
# Define the example to guide the model
formatted_prompt = f"""
Classify these reviews as Positive, Negative, or Mixed, and tag
relevant categories. Use this format:
- Sentiment: [Sentiment]
- Categories: [Categories].
{reviews_text}
"""
# Invoke the model with prompt
response = output.invoke(formatted_prompt)
# Print the response
print("Granite Model Formatted Response:\n")
print(response)